# Notebook for solving the Number Partitioning Problem
Given a set $S$ of numbers, partition the elements into 2 subsets such that the sum of each subset is as close as possible.

Solve using QUBOs and DWave's quantum annealer

In [1]:
# import necessary packages
import numpy as np
import numpy.random as random

import dimod

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import LazyEmbeddingComposite
from dwave import inspector

import time

## Function for solving QUBO
Choose either CPU or QPU solver

In [2]:
def solve_qubo(Q,
               sampler="CPU", # CPU or QPU
               k=10,
               chain_strength=None):
    """
    Given an upper triangular matrix Q of size NxN, solves the quadratic unconstrained binary
    optimization (QUBO) problem given by
    
        minimize sum(x[i] * Q[i,j] * x[j]
                     for i in range(N),
                     for j in range(i+1, N))
    
    Uses dimod.SimulatedAnnealingSampler, which solves the problem k times through simulated
    annealing (on a regular CPU). This method returns the best solution found.
    """
    assert isinstance(Q, np.ndarray)
    assert sampler in ["CPU", "QPU"]
    n = Q.shape[0]
    nz = len(Q[Q!=0])
    print("Solving QUBO problem (%d vars, %d nz) on %s..." % (n, nz, sampler))
    
    start = time.time()
    if sampler == "CPU":
        sampler = dimod.ExactSolver()
        response = sampler.sample_qubo(Q, num_reads=k)
    else:
        if chain_strength is None:
            chain_strength = int(10 * np.max(np.abs(Q)))
        sampler = LazyEmbeddingComposite(DWaveSampler(solver="Advantage_system6.1"))
        response = sampler.sample_qubo(Q, num_reads=k, chain_strength=chain_strength)
    elapsed = time.time() - start
    
    print("Solved in %.2f seconds" % elapsed)
    solution = min(response.data(["sample", "energy"]), key=lambda s: s.energy)
    return solution, response

## Create set of integers

In [3]:
n = 10
random.seed(42)
s = random.randint(11,size=(n))
print(s)

[ 6  3 10  7  4  6  9  2  6 10]


## Generate QUBO
Objective function: min(diff$^2=c^2-4x^TQx)$

$c$ is the sum of all the elements in $s$

Q is defined:
$q_{ii}=s_i(s_i-c) \quad q_{ij}=s_is_j$

In [4]:
c = np.sum(s)

Q = np.zeros([n,n])
for (x,y) in np.ndenumerate(Q):
    i = x[0]
    j = x[1]
    if i==j:
        Q[i][j] = s[i]*(s[i]-c)
    else:
        Q[i][j] = s[i]*s[j]
        
print("Q matrix: ")
print(Q)

Q matrix: 
[[-342.   18.   60.   42.   24.   36.   54.   12.   36.   60.]
 [  18. -180.   30.   21.   12.   18.   27.    6.   18.   30.]
 [  60.   30. -530.   70.   40.   60.   90.   20.   60.  100.]
 [  42.   21.   70. -392.   28.   42.   63.   14.   42.   70.]
 [  24.   12.   40.   28. -236.   24.   36.    8.   24.   40.]
 [  36.   18.   60.   42.   24. -342.   54.   12.   36.   60.]
 [  54.   27.   90.   63.   36.   54. -486.   18.   54.   90.]
 [  12.    6.   20.   14.    8.   12.   18. -122.   12.   20.]
 [  36.   18.   60.   42.   24.   36.   54.   12. -342.   60.]
 [  60.   30.  100.   70.   40.   60.   90.   20.   60. -530.]]


## Solve on classical CPU

In [5]:
k=10
# solve problem
solution, response = solve_qubo(Q,"CPU",k)

# display result
print(response)
# for smple, energy, num_occ in response.data(['sample','energy','num_occurrences']):
#     print(num_occ)

# display sums
print("")
S0 = [s[i] for (i, xi) in solution.sample.items() if xi >  0.5]
S1 = [s[i] for (i, xi) in solution.sample.items() if xi <= 0.5]
print("sum(S0) %8d" % sum(S0))
print("sum(S1) %8d" % sum(S1))

Solving QUBO problem (10 vars, 100 nz) on CPU...
Solved in 0.01 seconds
      0  1  2  3  4  5  6  7  8  9 energy num_oc.
53    1  1  1  1  0  1  0  0  0  0 -992.0       1
70    1  0  1  0  0  1  1  0  0  0 -992.0       1
72    0  0  1  1  0  1  1  0  0  0 -992.0       1
77    1  1  0  1  0  1  1  0  0  0 -992.0       1
81    1  0  0  1  1  1  1  0  0  0 -992.0       1
91    0  1  1  0  1  1  1  0  0  0 -992.0       1
101   1  1  1  0  1  0  1  0  0  0 -992.0       1
118   1  0  1  1  0  0  1  0  0  0 -992.0       1
139   0  1  1  1  0  0  1  1  0  0 -992.0       1
146   1  1  0  1  1  0  1  1  0  0 -992.0       1
151   0  0  1  1  1  0  1  1  0  0 -992.0       1
153   1  0  1  0  1  0  1  1  0  0 -992.0       1
167   0  0  1  0  1  1  1  1  0  0 -992.0       1
172   0  1  0  1  1  1  1  1  0  0 -992.0       1
201   1  0  1  1  0  1  0  1  0  0 -992.0       1
212   0  1  1  1  1  1  0  1  0  0 -992.0       1
218   1  1  1  0  1  1  0  1  0  0 -992.0       1
234   1  1  1  1  1  0  0  1

C:\Users\Josh\anaconda3\lib\site-packages\dimod\core\sampler.py:291: SamplerUnknownArgWarning: Ignoring unknown kwarg: 'num_reads'
  return self.sample(bqm, **parameters)


## Solve on a Quantum Annealer

In [6]:
k=1000
# solve problem
solution, response = solve_qubo(Q,"QPU",k)
print(solution)

Solving QUBO problem (10 vars, 100 nz) on QPU...
Solved in 1.80 seconds
Sample(sample={0: 1, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0, 8: 1, 9: 0}, energy=-992.0)


In [7]:
# display result
print(response)

# display sums
print("")
S0 = [s[i] for (i, xi) in solution.sample.items() if xi >  0.5]
S1 = [s[i] for (i, xi) in solution.sample.items() if xi <= 0.5]
print("sum(S0) %8d" % sum(S0))
print("sum(S1) %8d" % sum(S1))

     0  1  2  3  4  5  6  7  8  9 energy num_oc. chain_.
0    1  0  1  0  0  0  1  0  1  0 -992.0       3     0.0
1    1  1  1  0  1  0  1  0  0  0 -992.0       3     0.0
2    1  1  1  0  0  0  0  1  0  1 -992.0       2     0.0
3    1  0  1  0  1  0  0  1  0  1 -992.0       2     0.0
4    0  1  1  1  0  0  0  1  0  1 -992.0       1     0.0
5    0  0  1  0  0  1  1  0  1  0 -992.0       2     0.0
6    0  1  0  1  1  1  0  1  0  1 -992.0       1     0.0
7    1  0  1  0  0  1  1  0  0  0 -992.0       2     0.0
8    0  1  1  0  1  1  1  0  0  0 -992.0       8     0.0
9    0  1  1  0  0  0  0  1  1  1 -992.0       3     0.0
10   0  1  1  1  0  1  0  0  1  0 -992.0       2     0.0
11   0  1  1  0  1  0  1  0  1  0 -992.0       2     0.0
12   0  0  1  0  1  0  0  1  1  1 -992.0       1     0.0
13   0  1  0  1  0  0  1  1  0  1 -992.0       1     0.0
14   0  1  0  0  1  1  0  1  1  1 -992.0       1     0.0
15   1  1  1  0  0  1  0  0  1  0 -992.0       1     0.0
16   0  0  0  0  1  0  1  1  1 

In [8]:
inspector.show(response)

'http://127.0.0.1:18000/?problemId=6d2d7e45-a5af-4812-96be-d505dfa4fa13'